In [ ]:
#<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< INSTALLATION GUIDE>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 

'''pip install nltk

pip install spacy==2.3.5

pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz

pip install pyresparser'''

In [13]:
# import pdfplumber
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords 
from string import punctuation
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from heapq import nlargest
from collections import defaultdict
import pandas as pd 
from nltk.collocations import *
import base64,random

In [5]:
import os
import pandas as pd
from pyresparser import ResumeParser
import pdfplumber
import re

# Define the directory containing resumes
resumes_dir = 'cv_to_df/'
resumes_dir_full_path = os.path.abspath(resumes_dir)

def pdf_to_base64(file_path):
    
    encoded_pdf = base64.b64encode(file_path.read()).decode('utf-8')

    return encoded_pdf

def extract_text_from_pdf(filepath):
    with pdfplumber.open(filepath) as pdf:
        text = ""
        pages = pdf.pages
        for page in pages:
            text += page.extract_text(x_tolerance=2)
    return text

# Create an empty DataFrame object to store the extracted data
df = pd.DataFrame()

# Iterate through all files in the resumes directory
for filename in os.listdir(resumes_dir):
    if filename.endswith('.pdf'):  # Check if the file is a PDF
        # Apply the ResumeParser function to the current file and append the extracted data to the DataFrame
        data = ResumeParser(os.path.join(resumes_dir, filename)).get_extracted_data()
        # Add a new column to the DataFrame containing the file path
        data['file_path'] = os.path.join(resumes_dir, filename)
        # Append the extracted data to the DataFrame
        df = df.append(data, ignore_index=True)

# Extract text from all PDFs in the resumes directory and create a new DataFrame
data = []
for filename in os.listdir(resumes_dir):
    if filename.endswith('.pdf'):
        filepath = os.path.join(resumes_dir, filename)
        resume_data = extract_text_from_pdf(filepath)
        data.append({'file_path': filepath, 'All': resume_data})
df2 = pd.DataFrame(data)

# Merge the two data frames on the "file_path" column
merged_df = pd.merge(df, df2, on='file_path')
merged_df.drop(columns=["college_name","company_names","designation","experience","total_experience"], axis=1, inplace=True) # dropping unwanted col

# Define the regex pattern to match unwanted characters
unwanted_chars_pattern = r'[^a-zA-Z !@#$%&*_+-=|\:";<>,./()[\]{}\']'
for col in ['skills','degree', 'All']:
    merged_df[col] = merged_df[col].apply(lambda x: re.sub(unwanted_chars_pattern, '', str(x)))


merged_df


,degree,email,file_path,mobile_number,name,no_of_pages,skills,All
0,None,Jangrasukanya066@gmail.com,cv_to_df/3bdfa685-56f1-4f83-866c-dbb20be51048.pdf,None,Sukanya Jangra,1.0,"['Html', 'C++', 'Javascript', 'Excel', 'Word',...","Sukanya JangraBhiwani, HaryanaJangrasukanya066..."
1,"['Bachelor of Engineering,Computer Sciencenvis...",abhayownsthis@gmail.com,cv_to_df/abhay_kumar_visualcv_resume_with_GTC.pdf,9978045,Abhay Kumar,4.0,"['Caffe', 'Modeling', 'Website', 'Training', '...",Abhay KumarLead Data Scientist - Computer Visi...
2,"['Master of Engineering', 'Master in Computer ...",None,cv_to_df/Abhijit-Manepatil.pdf,None,Abhijit Manepatil,3.0,"['Modeling', 'English', 'Gis', 'Website', 'Alg...",Abhijit ManepatilData Scientist with Master in...
3,None,abhi4navv@gmail.com,cv_to_df/Abhinav_Abhishek_visualcv_resume (1).pdf,8757656959,Abhinav Abhishek,3.0,"['Warehouse', 'Training', 'Networking', 'Datab...",Abhinav AbhishekSenior Software Developer/Data...
4,"['Bachelor of Engineering, Computer Science n...",monu.aniket@gmail.com,cv_to_df/Aniket_cv.pdf,9691316944,Aniket Kashyap,3.0,"['Website', 'Algorithms', 'Database', 'Visual'...",Aniket KashyapSoftware Developer/Data Scientis...
5,None,anirudhanand80@gmail.com,cv_to_df/Anirudh_Anand_Resume.pdf,8082366408,Anirudh Anand,2.0,"['Seaborn', 'Pandas', 'Matplotlib', 'Analysis'...",Anirudh AnandMachine Learning Engineeranirudha...
6,['B.tech'],ashrayraina22@gmail.com,cv_to_df/ASHRAY_RAINA_Resume_13-02-2023-19-28-...,7780995937,ASHRAY RAINA,1.0,"['Statistical analysis', 'Seaborn', 'Matplotli...",ASHRAY RAINAashrayraina22@gmail.com 7780995937...
7,['Bachelors of Commerce'],Ganesh2480@gmail.com,cv_to_df/fe0e9156-9600-4679-a1cc-77cd4a2c122a.pdf,9820463399,Ramachandran Iyer,2.0,"['Website', 'Queries', 'Database', 'Design', '...",Ramachandran IyerIIM Ahmedabad Alumnus| SMP | ...
8,['Bachelors of Commerce'],Ganesh2480@gmail.com,cv_to_df/fe826f24-8900-4317-959f-cb4689540997.pdf,9820463399,Ramachandran Iyer,2.0,"['Website', 'Queries', 'Database', 'Design', '...",Ramachandran IyerIIM Ahmedabad Alumnus| SMP | ...
9,['BTech in Electronics'],None,cv_to_df/Himanshu-Tyagi.pdf,0800189689,Himanshu Tyagi,2.0,"['Presentation', 'Queries', 'Pattern', 'Datase...","Himanshu TyagiPune, Maharashtra - Email me on ..."


In [4]:
merged_df.columns

Index(['college_name', 'company_names', 'degree', 'designation', 'experience',
       'no_of_pages', 'total_experience', 'All'],
      dtype='object')

In [125]:
# df2 = merged_df.copy()

In [6]:
merged_df.to_csv('cv.csv')

In [3]:
df=pd.read_csv("cv.csv")

In [5]:
df.head()

,Unnamed: 0,degree,email,file_path,mobile_number,name,no_of_pages,skills,All
0,0,None,Jangrasukanya066@gmail.com,cv_to_df/3bdfa685-56f1-4f83-866c-dbb20be51048.pdf,NaN,Sukanya Jangra,1.0,"['Html', 'C++', 'Javascript', 'Excel', 'Word',...","Sukanya JangraBhiwani, HaryanaJangrasukanya066..."
1,1,"['Bachelor of Engineering,Computer Sciencenvis...",abhayownsthis@gmail.com,cv_to_df/abhay_kumar_visualcv_resume_with_GTC.pdf,9978045,Abhay Kumar,4.0,"['Caffe', 'Modeling', 'Website', 'Training', '...",Abhay KumarLead Data Scientist - Computer Visi...
2,2,"['Master of Engineering', 'Master in Computer ...",NaN,cv_to_df/Abhijit-Manepatil.pdf,NaN,Abhijit Manepatil,3.0,"['Modeling', 'English', 'Gis', 'Website', 'Alg...",Abhijit ManepatilData Scientist with Master in...
3,3,None,abhi4navv@gmail.com,cv_to_df/Abhinav_Abhishek_visualcv_resume (1).pdf,8757656959,Abhinav Abhishek,3.0,"['Warehouse', 'Training', 'Networking', 'Datab...",Abhinav AbhishekSenior Software Developer/Data...
4,4,"['Bachelor of Engineering, Computer Science n...",monu.aniket@gmail.com,cv_to_df/Aniket_cv.pdf,9691316944,Aniket Kashyap,3.0,"['Website', 'Algorithms', 'Database', 'Visual'...",Aniket KashyapSoftware Developer/Data Scientis...


In [30]:
import os
import pandas as pd
from pyresparser import ResumeParser
import pdfplumber
import re

In [35]:
def pdf_to_base64(file_path):
    #with pdfplumber.open(filepath) as pdf
    with open(file_path, 'rb') as f:
    
        encoded_pdf = base64.b64encode(f.read()).decode('utf-8')

    return encoded_pdf

In [36]:
encoded_files=[]
for i in df["file_path"].values:
    print()
    encoded_files.append(pdf_to_base64(i))

In [38]:
df["pdf_to_base64"]=encoded_files

In [39]:
df.to_csv("updated_cv.csv",index=False)